# 3. Evolve-Filter, time dep

* $\delta = 1/nx$
* $a = a_D$

In [4]:
%matplotlib inline 
import matplotlib
from dolfin import *
from __future__ import print_function
import numpy as np
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])

# Iliescu Exact Solution
c = 16.0*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
ue = c*h*(0.5+sym.atan(g)/sym.pi)

# ------------------------------------------ #

ue = sym.simplify(ue)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = sym.diff(ue,t)
fe += - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')

# print('u_code = ' + u_code + '\n')
# print('f_code = ' + f_code)

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	return L2n, H1n

def compute_extrema(u, t):
    maxval = np.amax(u.vector().get_local())
    minval = np.amin(u.vector().get_local())
    return maxval, minval

In [5]:
def EF_t(filename, delta, nx, dt, T, u_code, f_code, sigma, mu, velocity):
    degree = 2
    folder = 'FEFfigs/singlecodeN0/'

    
    t = 0.0
    u_exact = Expression(u_code, degree = degree, t = t)
    f = Expression(f_code, degree = degree, t = t)

    mesh = UnitSquareMesh(nx,nx)
    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree, t = t)
    
    def boundary(x, on_boundary):
        return on_boundary

    
# ------------ STEP 1: Solve on coarse grid ----------------------#

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
#     u_n = Function(Q)
    u_n = interpolate(u_D, Q)
    u_ = Function(Q)

    # Galerkin variational problem
    F = (u - u_n)*v*dx
    F += dt*(mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx)

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)

    # Outputting files
    #out_file_uexact = File(folder+"evolve_u_exact_"+str(nx)+".pvd") 
    #out_file_ubar = File(folder+str(mu)+"evolve_u_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    out_file_ind = File(folder+"EF_aD"+str(nx)+".pvd")
    out_file_ufilter = File(folder+"EF_u_"+str(nx)+".pvd") 

    # Save t = 0.0
    #out_file_uexact << (ue, float(t))
    
    
    u_.rename('u','u')
    indicator = Expression('0.0', degree=2, t=0.0)
    indicator = interpolate(indicator, Q)
    indicator.rename('a','a')
    
    out_file_ufilter << (u_, float(t))
    out_file_ind << (indicator, float(t))
    
    while t - T + dt < DOLFIN_EPS:
        # Step 1 Solve on Coarse Grid
        t += dt

        u_D.t = t
        f.t = t
        u_exact.t = t
        
        b = assemble(L)
        bc = DirichletBC(Q, u_D, boundary)
        bc.apply(A1)
        bc.apply(b)
        
        solve(A1, u_.vector(), b, 'gmres')
        progress.update(t / T)
        
#         if near(t,0.5):
#             errors = compute_errors(u_exact, u_, t, mesh)
#             print("Before Filter \n" + errors)

        #out_file_uexact << (ue, float(t))
        #out_file_ubar << (u_, float(t))
        
# ------------ STEP 2: Helmholtz filter ----------------------#
    
        # Helmholtz filter to compute the indicator function
        u_tilde = TrialFunction(Q)

        F_Hfilter = v*u_tilde*dx + delta*delta*dot(grad(v), grad(u_tilde))*dx - v*u_*dx 

        a_Hfilter = lhs(F_Hfilter)
        L_Hfilter = rhs(F_Hfilter)

        A_Hfilter = assemble(a_Hfilter)
        bc.apply(A_Hfilter)

        b_Hfilter = assemble(L_Hfilter)
        bc.apply(b_Hfilter)

        solver = LUSolver(A_Hfilter)
        u_tilde = Function(Q)
        solver.solve(u_tilde.vector(), b_Hfilter)

        # Compute the indicator function N = 0
        indicator = Expression('sqrt((a-b)*(a-b))', degree = 2, a = u_, b = u_tilde)
        indicator = interpolate(indicator, Q)
        max_ind = np.amax(indicator.vector().get_local())

        if max_ind < 1:
            max_ind = 1.0

        indicator = Expression('a/b', degree = 2, a = indicator, b = max_ind)
        indicator = interpolate(indicator, Q)

        indicator.rename('a','a')
        out_file_ind << (indicator, t)

        # Apply the filter
        u_bar = TrialFunction(Q)
        F_filter = v*u_bar*dx + delta*delta*dot(grad(v), indicator*grad(u_bar))*dx - v*u_*dx 

        a_filter = lhs(F_filter)
        L_filter = rhs(F_filter)

        A_filter = assemble(a_filter)
        bc.apply(A_filter)

        b_filter = assemble(L_filter)
        bc.apply(b_filter)

        solver = LUSolver(A_filter)
        u_bar = Function(Q)
        solver.solve(u_bar.vector(), b_filter)

        u_bar.rename('u','u')
        out_file_ufilter << (u_bar, float(t))

        u_n.assign(u_bar)
        
    L2, H1 = compute_errors(u_exact, u_bar, t, mesh)
    maxval, minval = compute_extrema(u_bar, t)
    
    print(nx,",",L2,",",H1,",",maxval,",",minval)
#     print("t = " + str(t))
#     errors = compute_errors(u_exact, u_, t, mesh)
#     print("\n Galerkin")
#     compute_extrema(u_, t)
#     ue = interpolate(u_exact, Q)
#     print("\n Exact ")
#     compute_extrema(ue, t)
#     print("\n" + errors)

## $\delta = 1/nx $

L2 = [0.281849030719,0.174892605469,0.029634489156,0.0051976175589,0.000418226657031,0];

H1 = [15.6615596083,12.864296479,7.49899879463,3.32468881229,0.643213010063,0];


In [3]:
filename = 'varies'
dt = 0.01
T = 0.5
print("nx L2 H1 maxval minval")
for nx in [25, 50, 100, 200, 400]:
    delta = 1.0/nx
    EF_t(filename, delta, nx, dt, T, u_code, f_code, sigma, mu, b)

nx L2 H1 maxval minval
25 , 0.113916088472 , 10.8946723868 , 1.44991375548 , -0.736454191268
50 , 0.0875243688834 , 12.5490676219 , 1.23982164053 , -0.467706228746
100 , 0.0205422669868 , 7.77171641781 , 1.02516060638 , -0.144339435904
200 , 0.00511608318586 , 3.05539895217 , 0.998160314884 , -0.0300812423359
400 , 0.00225944909481 , 1.13957685911 , 0.989085186034 , -0.00849326806143


## t=0.01, t=0.02

t=0.01

**before filter

L2gal = [0.00184782410145,0.0010581291447,0.000276852607002,4.6630251329e-05,5.6213014013e-06];

H1gal = [0.312363909881,0.332888253109,0.195881350549,0.0568651311196, 0.0124115252922];

**after filter

L2ef = [0.00158679828369,0.000946806949339,0.000258005764664,4.55552941288e-05,5.53278512514e-06];

H1ef = [0.25060047061,0.265846572127,0.172789388039,0.0542385133936, 0.0120017188395];

t=0.02

**before filter

L2gal = [0.00454904549291,0.00265823353184,0.000684438514297,0.000114081145201,1.5941938327e-05];

H1gal = [0.712570438202,0.789580523027,0.46713609235,0.131650758236,0.0258610593409];

**after filter

L2ef = [0.00363186332492,0.00225790566374,0.000613648532082,0.000110428702772,1.57643266427e-05];

H1ef = [0.502505766839,0.540745974632,0.379514644549,0.122441027543,0.0244500486396];


## t=0.5

L2ef = [0.100692482736,0.0790637680528,0.0194820200531, 0.00512982994389,0.00230586887979,0];

H1ef = [7.28040532884,6.18429343122,4.08087105438,2.45021640173,1.14766801288,0];

In [6]:
filename = 'varies'
dt = 0.01
T = 0.5
print("nx L2 H1 maxval minval")
for nx in [25, 50, 100, 200, 400]:
    delta = 1.0/nx
    EF_t(filename, delta, nx, dt, T, u_code, f_code, sigma, mu, b)

nx L2 H1 maxval minval
25 , 0.10044776823 , 7.28083849836 , 1.05742182053 , -0.233457353784


KeyboardInterrupt: 

* omg this code also had the mistake of calculating error wrong :/
 * Corrected error computation u_exact and u_bar